### 1. Import thư viện cần thiết

In [1]:
from pyspark.sql import SparkSession

### 2. Khởi tạo Spark

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("IE212_Lab2_Bai5") \
    .getOrCreate()
sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble
25/11/25 21:56:44 WARN Utils: Your hostname, ELUCIDATOR resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/25 21:56:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/25 21:56:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/25 21:56:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/25 21:56:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/11/25 21:56:48 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/11/25 21:56:48 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


### 3. Load dữ liệu

In [3]:
ratings1_path = "data/ratings_1.txt"
ratings2_path = "data/ratings_2.txt"
users_path = "data/users.txt"
occupations_path = "data/occupation.txt"

In [4]:
ratings_rdd = sc.textFile(f"{ratings1_path},{ratings2_path}")
users_rdd = sc.textFile(users_path)
occupations_rdd = sc.textFile(occupations_path)

### 4. Giải quyết bài toán

In [5]:
# 2. Xử lý Users: Lấy UserID và OccupationID
# Schema: UserID, Gender, Age, OccupationID, Zip-code
def parse_user_occ(line):
    parts = line.split(",")
    user_id = parts[0]
    occ_id = parts[3] # OccupationID nằm ở vị trí index 3
    return (user_id, occ_id)

user_occ_rdd = users_rdd.map(parse_user_occ)

In [6]:
# 3. Xử lý Ratings: Lấy UserID để Join
# Schema: UserID, MovieID, Rating, Timestamp
def parse_rating_simple(line):
    parts = line.split(",")
    user_id = parts[0]
    rating = float(parts[2])
    return (user_id, (rating, 1)) # (UserID, (Score, Count))

ratings_rdd_mapped = ratings_rdd.map(parse_rating_simple)

In [7]:
# 4. Join Users và Ratings (để gắn nghề nghiệp cho mỗi rating)
# user_occ_rdd: (UserID, OccID)
# ratings_rdd_mapped: (UserID, (Rating, 1))
# Kết quả Join: (UserID, (OccID, (Rating, 1)))
joined_user_rating = user_occ_rdd.join(ratings_rdd_mapped)

In [8]:
# 5. Tính toán thống kê theo OccupationID
# Bước này chuyển Key từ UserID sang OccupationID
def map_to_occ_stats(record):
    # record: (UserID, (OccID, (Rating, 1)))
    data = record[1]
    occ_id = data[0]
    rating_data = data[1] # (Rating, 1)
    return (occ_id, rating_data)

In [9]:
# reduceByKey: Cộng dồn điểm số và số lượt vote cho từng mã nghề
occ_stats = joined_user_rating.map(map_to_occ_stats).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

In [10]:
# 6. Xử lý bảng Occupation: Lấy ID và Tên nghề
# Schema: ID, OccupationName
def parse_occupation_name(line):
    parts = line.split(",")
    occ_id = parts[0]
    occ_name = parts[1]
    return (occ_id, occ_name)

occ_names_rdd = occupations_rdd.map(parse_occupation_name)

In [11]:
# 7. Join thống kê với Tên nghề nghiệp
# occ_names_rdd: (OccID, OccName)
# occ_stats: (OccID, (SumRating, TotalCount))
final_data = occ_names_rdd.join(occ_stats)

In [12]:
# 8. Format kết quả
def format_occ_result(record):
    # record: (OccID, (OccName, (SumRating, TotalCount)))
    data = record[1]
    occ_name = data[0]
    stats = data[1]
    
    total_rating = stats[0]
    count = stats[1]
    
    avg_rating = total_rating / count if count > 0 else 0
    
    return f"{occ_name} - AverageRating: {avg_rating:.2f} (TotalRatings: {count})"

In [13]:
# 9. Hiển thị kết quả
results = final_data.map(format_occ_result).collect()

for line in results:
    print(line)

Lawyer - AverageRating: 3.65 (TotalRatings: 17)
Manager - AverageRating: 3.47 (TotalRatings: 16)
Consultant - AverageRating: 3.86 (TotalRatings: 14)
Doctor - AverageRating: 3.69 (TotalRatings: 21)
Journalist - AverageRating: 3.85 (TotalRatings: 17)
Salesperson - AverageRating: 3.65 (TotalRatings: 17)
Designer - AverageRating: 4.00 (TotalRatings: 13)
Engineer - AverageRating: 3.56 (TotalRatings: 18)
Teacher - AverageRating: 3.70 (TotalRatings: 5)
Artist - AverageRating: 3.73 (TotalRatings: 11)
Accountant - AverageRating: 3.58 (TotalRatings: 6)
Student - AverageRating: 4.00 (TotalRatings: 8)
Programmer - AverageRating: 4.25 (TotalRatings: 10)
Nurse - AverageRating: 3.86 (TotalRatings: 11)
